In [4]:
import numpy as np

# define true and predicted values
y_true = np.array([10, 11, 12, 13])
y_pred = np.array([7,  9,  10, 11])

# calculate RMSE
rmse = np.sqrt(np.mean((y_pred - y_true)**2))

print("RMSE:", rmse)

# calculate percent error
target_range = np.max(y_true) - np.min(y_true)
percent_error = 100 * (rmse / target_range)

print("Percent Error:", percent_error)

RMSE: 2.29128784747792
Percent Error: 76.37626158259732


In [6]:
import pickle
with open('./biawebapp/dataP/forecast_LSTM.pkl', "rb") as file:
    forecast_LSTM = pickle.load(file)

In [3]:
import numpy as np

# define true and predicted values
y_true = np.array([100, 110, 120, 130])
y_pred = np.array([70,  90,  100, 110])

# calculate RMSE
rmse = np.sqrt(np.mean((y_pred - y_true)**2))

print("RMSE:", rmse)

# calculate percent error
target_range = np.max(y_true) - np.min(y_true)
percent_error = 100 * (rmse / target_range)

print("Percent Error:", percent_error)

RMSE: 22.9128784747792
Percent Error: 76.37626158259732
